In [1]:
import cv2
from mod import coor_check, sort_ano, read_anomalies, get_ano_info, play_ano, process_ano

ModuleNotFoundError: No module named 'mod'

In [2]:
# vid_no = 51
def read_anomalies(vid_no):
  text_file = open(f"D:\\bi12year3\intern\ictlab\imgonnacrylmao\\results\Output{vid_no}_anomalies.txt", "r")
  lines = text_file.readlines()
  no_anomalies = int(lines[-1][20:])

  ano = {}

  for i in range(no_anomalies):
      line = lines[i]
      line = line.strip().strip('()')
      key, value = line.split(',', 1)
      key = int(key.strip())
      value = eval(value.strip())
      
      # Assign the key-value pair to the dictionary
      ano[key] = value
  return ano

# anomailes = read_anomalies(vid_no)

# for i in anomailes.items():
#     print(i)

In [3]:

def sort_ano(anomailes: dict):
    ano = {}
    anomailes_cop = anomailes.copy()

    for ano_id1 in anomailes.keys():
        if ano_id1 in anomailes_cop.keys():
            x1, y1 = anomailes[ano_id1][:-3]
            start1 = anomailes[ano_id1][-2]
            end1 = anomailes[ano_id1][-1]
            ano[ano_id1] = [x1, y1, start1, end1]
            temp = []
            for ano_id2 in anomailes_cop.keys():
                if ano_id1 == ano_id2:
                    pass
                else:
                    x2, y2 = anomailes[ano_id2][:-3]
                    start2 = anomailes[ano_id2][-2]
                    end2 = anomailes[ano_id2][-1]

                    if coor_check(x1, y1, x2, y2, errors= 1):
                        ano[ano_id1] = [x1, y1, start1, end2]
                        temp.append(ano_id2)
            for i in temp:
                anomailes_cop.pop(i)  
    return ano

# sorted_anomalies = sort_ano(anomailes)


In [4]:
def get_ano_info(anomailes: dict, id):
    # print(anomailes[id])
    x, y = anomailes[id][:-2]
    start = anomailes[id][-2]
    end = anomailes[id][-1]
    return x, y, start, end

# for i in sorted_anomalies:
#     a = get_ano_info(sorted_anomalies, i)
#     print(a)


In [12]:
def display_ano(vid_no, ano_data):
  x, y, start, end = ano_data
  cap = cv2.VideoCapture(f"C:\\Users\\ADMIN\\Desktop\\AIC21-Track4-Anomaly-Detection_full\\AIC21-Track4-Anomaly-Detection\\aic21-track4-train-data\\{vid_no}.mp4")


  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  # Set the starting frame
  cap.set(cv2.CAP_PROP_POS_FRAMES, start[0])
  fps = 30
  # Read until video is completed
  while(cap.isOpened()):
    # Capture frame-by-frame
    ret, frame = cap.read()

    current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))

    if current_frame > end[0]:
        break

    timer = int(round(current_frame/fps, 0))
    if ret == True:
      # Display the resulting frame
      frame = cv2.putText(frame, '.', 
                          org= (x, y), 
                          fontFace= cv2.FONT_HERSHEY_COMPLEX,
                          fontScale= 1, 
                          color= (0, 0, 255), 
                          thickness= 2)
      
      frame = cv2.putText(frame, f'time: {timer}', 
                          org= (20, 50), 
                          fontFace= cv2.FONT_HERSHEY_COMPLEX,
                          fontScale= 1, 
                          color= (0, 255, 0), 
                          thickness= 1)

      cv2.imshow(f'frame{vid_no}',frame)
  
      # Press Q on keyboard to  exit
      if cv2.waitKey(25) & 0xFF == ord('q'):
        break
      
      # Break the loop
      # else: 
      #   break

  
  # When everything done, release the video capture object
  cap.release()
  
  # Closes all the frames
  cv2.destroyAllWindows()

In [6]:
# print(sorted_anomalies)
# for i in sorted_anomalies:
#     display_ano(vid_no, get_ano_info(sorted_anomalies, i))
    

In [2]:
def process_ano(vid_no):
    return sort_ano(anomailes= read_anomalies(vid_no))

def play_ano(vid_no, sorted_ano: dict):
    print("Number of anomalies: ", len(sorted_ano.keys()) )
    for i, ano_id in enumerate(sorted_ano.items()):
        print(f"Anomaly {i+1}")
        print(ano_id)
        display_ano(vid_no, get_ano_info(sorted_ano, ano_id[0]))

In [6]:
def display_ano(vid_no, ano_data):
    coors, start, end = ano_data
    cap = cv2.VideoCapture(f"C:\\Users\\ADMIN\\Desktop\\AIC21-Track4-Anomaly-Detection_full\\AIC21-Track4-Anomaly-Detection\\aic21-track4-train-data\\{vid_no}.mp4")
    x1, x2, y1, y2 = coors
    
    # Check if camera opened successfully
    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
    fps = 30
    # Set the starting frame
    cap.set(cv2.CAP_PROP_POS_FRAMES, start[0])
    # Read until video is completed
    while(cap.isOpened()):
    # Capture frame-by-frame
        ret, frame = cap.read()

        current_frame = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
        
        if current_frame > end[0]:
            break

        timer = int(round(current_frame/fps, 0))

        if ret == True:
            # Display the resulting frame
            frame = cv2.rectangle(frame, 
                                pt1=(x1, x2), 
                                pt2= (y1, y2),  
                                color= (0, 0, 255), thickness= 2)
            
            frame = cv2.putText(frame, f'time: {timer}', 
                          org= (20, 50), 
                          fontFace= cv2.FONT_HERSHEY_COMPLEX,
                          fontScale= 1, 
                          color= (0, 255, 0), 
                          thickness= 1)

            cv2.imshow(f'frame{vid_no}',frame)

            # Press Q on keyboard to  exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        
        # Break the loop
        # else: 
        #   break


    # When everything done, release the video capture object
    cap.release()

    # Closes all the frames
    cv2.destroyAllWindows()

In [5]:
vid_no = 2
# print(read_anomalies(vid_no))
# print("************************")
# hihi = sort_ano(read_anomalies(vid_no))
# print(len(hihi.items()))
# hihi
process_ano(vid_no)
# print(process_ano(vid_no))



{179: [[6, 95, 44, 117], [914, 30], [26399, 880]],
 539: [[5, 96, 43, 113], [2218, 74], [16497, 550]],
 5067: [[634, 78, 649, 88], [18770, 626], [26819, 894]],
 5200: [[585, 94, 604, 109], [19276, 643], [26819, 894]]}

In [ ]:
play_ano(vid_no, process_ano(vid_no))